In [2]:
import autokeras as ak
from keras.layers import Dense, LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import tensorflow as tf
import reber
RANDOM_STATE = 42
PADDING_IDX = 0

Using TensorFlow backend.


In [27]:
r = reber.ReberGenerator(max_length=15)
X, y = r.make_data(50000)
_, word_len = X.shape
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.15, random_state=RANDOM_STATE
)

In [25]:
X.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
count,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.000000,120000.00000
mean,1.167342,4.001592,1.165125,3.974617,5.467817,5.190667,4.334725,4.184042,3.561342,2.449292,1.665400,1.104875,0.623558,0.370683,0.10195
std,0.835490,1.082135,0.828863,1.079428,1.229692,1.455466,1.868470,1.651977,1.719170,2.098686,2.006475,1.727532,1.289127,1.075308,0.44220
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1.000000,3.000000,1.000000,3.000000,5.000000,4.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,1.000000,4.000000,1.000000,3.000000,6.000000,6.000000,4.000000,4.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,1.000000,5.000000,1.000000,5.000000,7.000000,6.000000,6.000000,6.000000,5.000000,4.000000,3.000000,2.000000,0.000000,0.000000,0.00000
max,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.00000


In [28]:
num_features = len(r._reber_letters) + 1 # include padding token

input_node = ak.Input()
output_node = ak.Embedding(# TODO: how to mask in the embedding
    max_features=num_features,
    pretraining='random',
    dropout_rate=0
)(input_node)
output_node = ak.RNNBlock(
    bidirectional=False,
#     num_layers=1, # let it search the space
#     layer_type='gru'
)(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    max_trials=8
)
# TODO: increase number of epochs
clf.fit(X_train.to_numpy(), y_train.to_numpy(), verbose=2, epochs=10)

INFO:tensorflow:Reloading Oracle from existing project ./auto_model/oracle.json
INFO:tensorflow:Reloading Tuner from ./auto_model/tuner0.json
Train for 1063 steps, validate for 266 steps
Epoch 1/10
1063/1063 - 82s - loss: 0.6699 - accuracy: 0.5347 - val_loss: 0.6551 - val_accuracy: 0.5585
Epoch 2/10
1063/1063 - 83s - loss: 0.6583 - accuracy: 0.5480 - val_loss: 0.6588 - val_accuracy: 0.5531
Epoch 3/10
1063/1063 - 70s - loss: 0.6597 - accuracy: 0.5448 - val_loss: 0.6587 - val_accuracy: 0.5529
Epoch 4/10
1063/1063 - 64s - loss: 0.6596 - accuracy: 0.5468 - val_loss: 0.6579 - val_accuracy: 0.5542
Epoch 5/10
1063/1063 - 64s - loss: 0.6593 - accuracy: 0.5468 - val_loss: 0.6585 - val_accuracy: 0.5533
Epoch 6/10
1063/1063 - 63s - loss: 0.6616 - accuracy: 0.5454 - val_loss: 0.6604 - val_accuracy: 0.5521
Epoch 7/10
1063/1063 - 61s - loss: 0.6539 - accuracy: 0.5556 - val_loss: 0.6428 - val_accuracy: 0.5751
Epoch 8/10
1063/1063 - 63s - loss: 0.6427 - accuracy: 0.5678 - val_loss: 0.6426 - val_accura

Train for 1063 steps, validate for 266 steps
Epoch 1/10
1063/1063 - 73s - loss: 0.6606 - accuracy: 0.5478 - val_loss: 0.5455 - val_accuracy: 0.7114
Epoch 2/10
1063/1063 - 66s - loss: 0.1213 - accuracy: 0.9622 - val_loss: 0.0533 - val_accuracy: 0.9858
Epoch 3/10
1063/1063 - 68s - loss: 0.0680 - accuracy: 0.9836 - val_loss: 0.0605 - val_accuracy: 0.9852
Epoch 4/10
1063/1063 - 69s - loss: 0.0844 - accuracy: 0.9784 - val_loss: 0.0515 - val_accuracy: 0.9885
Epoch 5/10
1063/1063 - 63s - loss: 0.0675 - accuracy: 0.9836 - val_loss: 0.0487 - val_accuracy: 0.9887
Epoch 6/10
1063/1063 - 62s - loss: 0.0649 - accuracy: 0.9836 - val_loss: 0.0353 - val_accuracy: 0.9915
Epoch 7/10
1063/1063 - 63s - loss: 0.0458 - accuracy: 0.9885 - val_loss: 0.0431 - val_accuracy: 0.9907
Epoch 8/10
1063/1063 - 62s - loss: 0.0333 - accuracy: 0.9929 - val_loss: 0.0275 - val_accuracy: 0.9946
Epoch 9/10
1063/1063 - 59s - loss: 0.0384 - accuracy: 0.9910 - val_loss: 0.0367 - val_accuracy: 0.9905
Epoch 10/10
1063/1063 - 65s 

INFO:tensorflow:Oracle triggered exit
Train for 1329 steps, validate for 266 steps
Epoch 1/10
1329/1329 - 89s - loss: 0.6610 - accuracy: 0.5492 - val_loss: 0.6530 - val_accuracy: 0.5631
Epoch 2/10
1329/1329 - 77s - loss: 0.6515 - accuracy: 0.5590 - val_loss: 0.6461 - val_accuracy: 0.5701
Epoch 3/10
1329/1329 - 82s - loss: 0.6486 - accuracy: 0.5625 - val_loss: 0.6467 - val_accuracy: 0.5686
Epoch 4/10
1329/1329 - 78s - loss: 0.2888 - accuracy: 0.8532 - val_loss: 0.1026 - val_accuracy: 0.9739
Epoch 5/10
1329/1329 - 77s - loss: 0.0819 - accuracy: 0.9797 - val_loss: 0.0889 - val_accuracy: 0.9785
Epoch 6/10
1329/1329 - 77s - loss: 0.0654 - accuracy: 0.9842 - val_loss: 0.0418 - val_accuracy: 0.9908
Epoch 7/10
1329/1329 - 76s - loss: 0.0536 - accuracy: 0.9873 - val_loss: 0.0388 - val_accuracy: 0.9922
Epoch 8/10
1329/1329 - 83s - loss: 0.0350 - accuracy: 0.9921 - val_loss: 0.0255 - val_accuracy: 0.9952
Epoch 9/10
1329/1329 - 81s - loss: 0.0332 - accuracy: 0.9921 - val_loss: 0.0108 - val_accurac

In [29]:
clf.evaluate(X_test.to_numpy(), y_test.to_numpy())

235/235 [==============================] - ETA: 3:33 - loss: 0.0016 - accuracy: 1.00 - ETA: 31s - loss: 0.0021 - accuracy: 1.0000 - ETA: 20s - loss: 0.0080 - accuracy: 0.997 - ETA: 17s - loss: 0.0177 - accuracy: 0.995 - ETA: 13s - loss: 0.0214 - accuracy: 0.994 - ETA: 12s - loss: 0.0186 - accuracy: 0.995 - ETA: 11s - loss: 0.0166 - accuracy: 0.996 - ETA: 10s - loss: 0.0151 - accuracy: 0.996 - ETA: 10s - loss: 0.0147 - accuracy: 0.996 - ETA: 9s - loss: 0.0138 - accuracy: 0.996 - ETA: 9s - loss: 0.0128 - accuracy: 0.99 - ETA: 8s - loss: 0.0118 - accuracy: 0.99 - ETA: 8s - loss: 0.0111 - accuracy: 0.99 - ETA: 7s - loss: 0.0105 - accuracy: 0.99 - ETA: 7s - loss: 0.0128 - accuracy: 0.99 - ETA: 7s - loss: 0.0124 - accuracy: 0.99 - ETA: 6s - loss: 0.0119 - accuracy: 0.99 - ETA: 6s - loss: 0.0113 - accuracy: 0.99 - ETA: 6s - loss: 0.0107 - accuracy: 0.99 - ETA: 5s - loss: 0.0105 - accuracy: 0.99 - ETA: 5s - loss: 0.0101 - accuracy: 0.99 - ETA: 5s - loss: 0.0142 - accuracy: 0.99 - ETA: 5s - los

[0.014766630970506353, 0.9964]

In [30]:
model = clf.export_model()

In [31]:
model.save('99_test_acc.h5')

In [25]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=1,
    batch_size=5000
)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 80000 samples, validate on 20000 samples
Epoch 1/2
80000/80000 [==============================] - 17s 217us/step - loss: 0.6903 - accuracy: 0.5657 - val_loss: 0.6851 - val_accuracy: 0.4983
Epoch 2/2
80000/80000 [==============================] - 18s 224us/step - loss: 0.6665 - accuracy: 0.5379 - val_loss: 0.6395 - val_accuracy: 0.6458


In [40]:
# strs = [
#     "BPBTSSXXVVEPE", # true reber
#     "BPBTSSSSXXVPXTTTVPSETE", # second and second to last not the same, but reber inbetween
#     "BPBPTTTTTVPBTTTTVPSEPT", # second and second to last the same, but not reber in middle
#     "BBBBBBBBBBB",
#     "XXXXXXXXXXXXXX",
#     "TTTTTTTTTTTTTTTTTTT"
# ]
x = np.array([
        r.encode_as_padded_ints(s) for s in ['BPBTSSXXVVEPPE']
    ])
dataset = clf._process_x(x, False)
dataset = dataset.batch(6)
model = clf.tuner.get_best_model()
model.predict(x)

array([[0.97984904]], dtype=float32)

Clearly, XXXXXXXXXX is not in the reber grammar, and yet the model predicted with 98.6% certainty that it was. This means that I need to make my examples of non-reber much more random.

In [24]:
## autokeras
embedding_vector_length = len(r._reber_letters) # TODO: what should this be?
num_neurons = 60
model = Sequential()
Embedding
model.add(
    Embedding(
        embedding_vector_length + 1,
        embedding_vector_length,
        input_length=word_len,
        mask_zero=True
    )
)
model.add(LSTM(num_neurons))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])